In [1]:
using LinearAlgebra
using Polyhedra

using CDDLib
using Clarabel, Ipopt

using TropicalFrechetMeans

In [2]:
function trop_normalize(x)
    return x .- first(x)
end

trop_normalize (generic function with 1 method)

Here, we define the sample. For this, we need to use `BigInt` as the underlying scalar type because some computations below overflow otherwise.

In [3]:
sample = map([
    [2//10, 4//10, 2//1, 4//10, 2//1, 2//1],
    [2//1, 2//1, 2//1, 4//10, 4//10, 2//1],
    [4//10, 4//10, 2//1, 2//10, 2//1, 2//1]
]) do pt; Rational{BigInt}.(pt) end

3-element Vector{Vector{Rational{BigInt}}}:
 [1//5, 2//5, 2, 2//5, 2, 2]
 [2, 2, 2, 2//5, 2//5, 2]
 [2//5, 2//5, 2, 1//5, 2, 2]

## Clarabel

In [4]:
FM_clarabel = tropical_frechet_mean(sample; tol=0)

6-element Vector{Float64}:
 -0.7725470731887855
 -0.7725470115433544
  0.03351399424026832
 -1.048104444707801
 -0.1724770004909199
  0.03351399424027505

The following calculation suggests that for a tropical Fréchet mean the sum of square distances to the sample is close to $7.28$.

In [5]:
sos_clarabel = sum_of_trop_dist(FM_clarabel, sample; power=2) 

7.280000138012775621504297395663831911704951328104053148208134281223029293794702

We can also calculate the FM polytrope to obtain more Fréchet means.

In [6]:
P_clarabel = tropical_frechet_set(sample)

Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
10-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, -1, 0, 0], 1//1)
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, -1, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, -1, 0, 0, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, -1, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, -1], -3//5)
 HalfSpace(Rational{BigInt}[0, 0, 1, 0, -1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 0, 0, 1, -1, 0], -4//5)
 HalfSpace(Rational{BigInt}[-1, 0, 0, 0, 1, 0], 3//5)
 HalfSpace(Rational{BigInt}[0, -1, 0, 0, 1, 0], 3//5)
 HalfSpace(Rational{BigInt}[0, 0, 0, 0, -1, 1], 1//1)

In [7]:
pts_clarabel = points(P_clarabel)

8-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-3//5, -3//5, 0, -4//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -4//5, 0, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -9//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -9//5, -1, 0]

Using `Clarabel`, we obtain a FM polytrope $\overline{P}$ whose vertices all attain the same sum of square distances to the sample.

In [8]:
sum_of_trop_dist.(pts_clarabel, Ref(sample); power=2)

8-element Vector{Rational{BigInt}}:
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25

In particular, the vertices of $\overline{P}$ realize a sum of square distances of exactly $7.28$.

In [9]:
sum_of_trop_dist(pts_clarabel |> first, sample; power=2) |> Float64

7.28

## Ipopt

In [10]:
FM_ipopt = tropical_frechet_mean(Ipopt.Optimizer, sample; tol=0)

6-element Vector{Float64}:
 -0.5665307602267998
 -0.5665307577476556
  0.6262037885666266
 -1.0869983660316738
  0.0334691296903913
  0.6262037885666267

The tropical FM we obtain from `Ipopt` beahves similarly to the one from `Clarabel`. 

In [11]:
sos_ipopt = sum_of_trop_dist(FM_ipopt, sample; power=2) 

7.280000004958325353131174279587538977394609420973774267107165773981944639075998

In [12]:
P_ipopt = tropical_frechet_set(Ipopt.Optimizer, sample);

In [13]:
pts_ipopt = points(P_ipopt)

8-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-8//5, -8//5, -1, -9//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -9//5, -1, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -4//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -4//5, 0, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -13//5, -1, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -8//5, 0, 0]

In [14]:
sum_of_trop_dist.(pts_ipopt, Ref(sample); power=2)

8-element Vector{Rational{BigInt}}:
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25